# Feature Engineering

## 特征工程的目标

特征工程的目标很简单：就是为了使您的数据更适合机器学习模型的特征。这可能意味着让您的数据更易于预测，也可能意味着使您的数据更易于理解。


你可能会进行特征工程来：

- 改进模型的预测性能
- 减少计算或数据需求
- 提高结果的可解释性

## 特征工程的指导原则

要使一个特征有用，它必须与你的模型要学习的目标之间存在某种关系。例如，线性模型只能学习线性关系。因此，在使用线性模型时，你的目标是转换特征，使其与目标的关系线性化。





# 1. BaseLine Model
BaseLine Model是指在没有进行任何特征工程的情况下，直接使用原始数据进行建模，得到的模型称为BaseLine Model。
一般来说，我们的目标是在BaseLine Model的基础上，通过特征工程，使得模型的效果有所提升。

Baseline Model一般使用最简单的模型，比如线性回归、逻辑回归、决策树等等。当模型完成后，使用一个评价指标来评价模型的效果，比如回归问题中常用的MSE、MAE、R2等，分类问题中常用的AUC、KS、F1等。

在得到Baseline Model的效果后，我们就可以进行特征工程，将特征进行组合、转换、筛选等等，然后再次训练模型，得到新的模型效果。如果新的模型效果比Baseline Model的效果要好，那么说明我们的特征工程是有效的。

在进行特征工程和模型优化时，是否应该保持与基线模型相同的算法，这取决于你的目标和实验设计。通常，有两种主要的方法：

1. **保持相同算法，改变特征选择**：这种方法的优点在于它可以让你直接评估特征工程的效果。如果你在特征工程之后仍然使用随机森林算法，那么任何性能上的提升或下降都可以直接归因于新特征的影响。这样做有助于理解哪些特征是有用的，哪些可能不是。

2. **改变算法和特征选择**：另一种方法是在特征工程之后尝试不同的算法。这种方法可以帮助你找到最适合新特征集的模型。有时，新的特征可能更适合不同类型的算法。例如，一些特征可能对某些模型（如基于树的模型）更有效，而对其他模型（如线性模型）效果不佳。

在实际操作中，你可以先采用第一种方法，只改变特征选择，以评估这些新特征对模型性能的影响。一旦确定了有效的特征集，再尝试不同的算法，以找到最佳的模型配置。这种迭代方法可以帮助你更全面地理解数据，并找到最优的模型。